In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd ./drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


### インストール

In [2]:
!pip install cshogi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 34.9 MB/s 


In [ ]:
# ソースコードのクローン
!git clone https://github.com/TadaoYamaoka/python-dlshogi2.git

Cloning into 'python-dlshogi2'...
remote: Enumerating objects: 209, done.
remote: Counting objects: 100% (16/16), done.
remote: Total 209 (delta 15), reused 15 (delta 15), pack-reused 193
Receiving objects: 100% (209/209), 313.81 MiB | 16.15 MiB/s, done.
Resolving deltas: 100% (112/112), done.
Checking out files: 100% (23/23), done.


In [3]:
# パッケージインストール
%cd python-dlshogi2
!pip install -e .

/content/drive/MyDrive/python-dlshogi2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/python-dlshogi2
  Running setup.py develop for python-dlshogi2


## モデルの学習

### 訓練データの準備

#### floodgateの棋譜

In [ ]:
%cd /content/drive/MyDrive/python-dlshogi2/floodgate
!7z x wdoor2016.7z

/content/drive/MyDrive/python-dlshogi2/floodgate


In [4]:
# ダウンロード
%cd /content
!mkdir floodgate
%cd floodgate
!wget 'https://osdn.net/frs/redir.php?m=iij&f=shogi-server%2F68500%2Fwdoor2016.7z' -O wdoor2016.7z
!7z x wdoor2016.7z

Output hidden; open in https://colab.research.google.com to view.

In [5]:
!wget 'http://wdoor.c.u-tokyo.ac.jp/shogi/x/wdoor2017.7z' -O wdoor2017.7z

--2022-10-18 02:54:02--  http://wdoor.c.u-tokyo.ac.jp/shogi/x/wdoor2017.7z
Resolving wdoor.c.u-tokyo.ac.jp (wdoor.c.u-tokyo.ac.jp)... 157.82.29.44
Connecting to wdoor.c.u-tokyo.ac.jp (wdoor.c.u-tokyo.ac.jp)|157.82.29.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 192495725 (184M) [application/x-7z-compressed]
Saving to: ‘wdoor2017.7z’

wdoor2017.7z        100%[===================>] 183.58M  37.1MB/s    in 5.4s    

2022-10-18 02:54:08 (33.7 MB/s) - ‘wdoor2017.7z’ saved [192495725/192495725]



In [6]:
!7z x wdoor2017.7z

Output hidden; open in https://colab.research.google.com to view.

#### CSA形式の棋譜からhcpe形式に変換

In [ ]:
%cd /content/drive/MyDrive/python-dlshogi2
!python utils/csa_to_hcpe.py /content/drive/MyDrive/python-dlshogi2/floodgate train.hcpe test.hcpe

In [7]:
%cd /content/drive/MyDrive/python-dlshogi2
!python utils/csa_to_hcpe.py /content/floodgate train.hcpe test.hcpe

/content/drive/MyDrive/python-dlshogi2
kif_num 4830
position_num 787638
kif_num 555
position_num 90910


### 訓練の実行

In [ ]:
!python pydlshogi2/train.py --epoch 100 --batchsize 64 train.hcpe test.hcpe

2022/10/18 03:19:57	INFO	batchsize=64
2022/10/18 03:19:57	INFO	lr=0.01
2022/10/18 03:20:01	INFO	Reading training data
2022/10/18 03:20:01	INFO	train.hcpe
2022/10/18 03:20:01	INFO	Reading test data
2022/10/18 03:20:01	INFO	test.hcpe
2022/10/18 03:20:01	INFO	train position num = 787638
2022/10/18 03:20:01	INFO	test position num = 90910
2022/10/18 03:20:15	INFO	epoch = 1, steps = 100, train loss = 6.9708000, 0.6742312, 7.6450312, test loss = 6.2020636, 0.6625866, 6.8646501, test accuracy = 0.0742188, 0.5634766
2022/10/18 03:20:20	INFO	epoch = 1, steps = 200, train loss = 6.0506397, 0.6536898, 6.7043295, test loss = 5.8653274, 0.6679107, 6.5332381, test accuracy = 0.0791016, 0.5761719
2022/10/18 03:20:25	INFO	epoch = 1, steps = 300, train loss = 5.5284426, 0.6487215, 6.1771641, test loss = 5.2757473, 0.6335313, 5.9092786, test accuracy = 0.1455078, 0.6064453
2022/10/18 03:20:31	INFO	epoch = 1, steps = 400, train loss = 5.2223861, 0.6469074, 5.8692934, test loss = 4.9320269, 0.6478263, 5.57

In [ ]:
# チェックポイントが保存されていることを確認
!ls -l checkpoints

total 405157
-rw------- 1 root root 59268253 Oct 13 06:40 checkpoint-100.pth
-rw------- 1 root root 59268253 Oct 13 07:02 checkpoint-150.pth
-rw------- 1 root root 59268253 Oct 13 07:24 checkpoint-200.pth
-rw------- 1 root root 59268253 Oct 13 07:46 checkpoint-250.pth
-rw------- 1 root root 59268253 Oct 13 07:59 checkpoint-280.pth
-rw------- 1 root root 59268253 Oct 13 08:04 checkpoint-290.pth
-rw------- 1 root root 59268253 Oct 13 08:08 checkpoint-300.pth


#### チェックポイントをローカルにダウンロード

In [ ]:
# zipで圧縮
!zip checkpoint-100.zip checkpoints/checkpoint-100.pth

左のファイルタブから、python-dlshogi2/checkpoint-001.zipを右クリックしてダウンロードを選択します。

## 対局プログラムの実装

### テスト

In [ ]:
# ディレクトリ移動
%cd /content/drive/MyDrive/python-dlshogi2

In [ ]:
# 作成済みのmcts_player.shの内容表示
!cat mcts_player.sh

In [ ]:
# 実行権を付与
!chmod +x mcts_player.sh

- setoption name debug value true
- setoption name modelfile value checkpoints/checkpoint-001.pth
- isready
- readyok
- position startpos
- go byoyomi 1000
- quit

In [ ]:
# 実行
!./mcts_player.sh

## Colabで対局

In [ ]:
# パッケージをインポート
import cshogi.web.app

In [ ]:
!pip install cshogi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 人間プレイヤーとの対局

In [ ]:
# 人間プレイヤーと対局
cshogi.web.app.colab(
    engine1='human',
    engine2='./mcts_player.sh',
    options2={'modelfile': 'checkpoints/checkpoint-300.pth'},
    byoyomi=1000)

### USIエンジンとの対局

In [ ]:
# Lesserkaiのソースをダウンロードしてビルド
%cd /content/
!wget http://shogidokoro.starfree.jp/download/LesserkaiSrc.zip
!unzip LesserkaiSrc.zip
%cd LesserkaiSrc/Lesserkai
!make

/content
--2022-10-13 08:22:52--  http://shogidokoro.starfree.jp/download/LesserkaiSrc.zip
Resolving shogidokoro.starfree.jp (shogidokoro.starfree.jp)... 157.112.187.101
Connecting to shogidokoro.starfree.jp (shogidokoro.starfree.jp)|157.112.187.101|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47737 (47K) [application/zip]
Saving to: ‘LesserkaiSrc.zip’

LesserkaiSrc.zip    100%[===================>]  46.62K   292KB/s    in 0.2s    

2022-10-13 08:22:52 (292 KB/s) - ‘LesserkaiSrc.zip’ saved [47737/47737]

Archive:  LesserkaiSrc.zip
  inflating: LesserkaiSrc/Lesserkai/Globals.cpp  
  inflating: LesserkaiSrc/Lesserkai/Joseki.cpp  
  inflating: LesserkaiSrc/Lesserkai/KomaMoves.cpp  
  inflating: LesserkaiSrc/Lesserkai/Kyokumen.cpp  
  inflating: LesserkaiSrc/Lesserkai/Kyokumen.h  
  inflating: LesserkaiSrc/Lesserkai/Lesserkai.cpp  
  inflating: LesserkaiSrc/Lesserkai/Lesserkai.vcxproj  
  inflating: LesserkaiSrc/Lesserkai/Lesserkai.vcxproj.filters  
  inflating:

In [ ]:
# ディレクトリを元の場所に移動
%cd /content/drive/MyDrive/python-dlshogi2


/content/drive/MyDrive/python-dlshogi2


In [ ]:
# Lesserkaiと対局
cshogi.web.app.colab(
    engine1='./mcts_player.sh',
    engine2='/content/LesserkaiSrc/Lesserkai/Lesserkai',
    options1={'modelfile': 'checkpoints/checkpoint-300.pth'},
    byoyomi=1000)

<IPython.core.display.Javascript object>

### dlshogiのモデルと対局

In [ ]:
# onnxruntimeをインストール
!pip install onnxruntime-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 111.0 MB 48 kB/s 
     |████████████████████████████████| 46 kB 4.3 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 


In [ ]:
# 作成済みのonnx_player.shの内容表示
!cat onnx_player.sh

#!/bin/sh
python -m pydlshogi2.player.onnx_player


In [ ]:
# 実行権を付与
!chmod +x onnx_player.sh

In [ ]:
# GCT電竜のモデルと対局
cshogi.web.app.colab(
    engine1='./mcts_player.sh',
    engine2='./onnx_player.sh',
    options1={'modelfile': 'checkpoints/checkpoint-200.pth'},
    options2={'modelfile': 'model/model-0000167.onnx'},
    byoyomi=1000)